##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# ローカルオーケストレーターでテンプレートを使用して TFX パイプラインを作成する

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/template_local"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で実行</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template_local.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colabで実行</a>
</td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/template_local.ipynb">     <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/template_local.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table></div>

## 重み値のみを保存。（通常、モデルのトレーニング時に使用）。

このドキュメントでは、TFX Python パッケージで提供される*テンプレート*を使用して TensorFlow Extended（TFX）パイプラインを作成する手順について説明します。命令の多くは Linux シェルコマンドであり、`!` を使用してそれらのコマンドを呼び出す、対応する Jupyter Notebook コードセルが提供されています。

シカゴ市が公開した[タクシー乗降データセット](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)を使用してパイプラインを構築します。このパイプラインをベースラインとして利用し、データセットを使用して独自のパイプラインを構築してみることを強くお勧めします。

ローカル環境で実行されるパイプラインを構築します。Google Cloud で Kubeflow オーケストレーターを使用することに興味がある場合は、[クラウド AI プラットフォームパイプラインでの TFX チュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines)を参照してください。

## 前提条件

- Linux / MacOS
- Python &gt;= 3.5.3

[このノートブックを Google Colab で実行](https://colab.sandbox.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template_local.ipynb)すると、すべての前提条件を簡単に満たすことができます。


## ステップ 1.  環境をセットアップする

**このドキュメント全体を通して、コマンドを 2 回示します。コピーアンドペースト対応のシェルコマンドとして 1 回、Jupyter ノートブックセルとして 1 回です。Colab を使用している場合は、シェルスクリプトブロックをスキップしてノートブックセルを実行するだけです。**

パイプラインを構築するための開発環境を準備する必要があります。

`tfx` パッケージをインストールします。ローカル環境で `virtualenv` を使用することをお勧めします。次のシェルスクリプトスニペットを使用して、環境を設定できます。

```sh
# Create a virtualenv for tfx.
virtualenv -p python3 venv
source venv/bin/activate
# Install python packages.
python -m pip install --upgrade "tfx<2"
```

colab を使用している場合


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade "tfx<2"

注意: パッケージのインストール中にエラーが発生する可能性があります。例えば、

> ERROR: some-package 0.some_version.1 has requirement other-package!=2.0.,&lt;3,&gt;=1.15, but you'll have other-package 2.0.0 which is incompatible.

現時点では、これらのエラーは無視してください。

In [ ]:
# Set `PATH` to include user python binary directory.
HOME=%env HOME
PATH=%env PATH
%env PATH={PATH}:{HOME}/.local/bin

TFX のバージョンを確認しましょう。

```bash
python -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"
```

In [ ]:
!python3 -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"

以上でパイプラインを作成する準備ができました。

## ステップ 2. 事前定義されたテンプレートをプロジェクトディレクトリにコピーする

このステップでは、事前定義されたテンプレートから追加のファイルをコピーして、作業パイプラインプロジェクトディレクトリとファイルを作成します。

以下の `PIPELINE_NAME` を変更することで、パイプラインに別の名前を付けることができます。また、これはファイルが配置されるプロジェクトディレクトリ名にもなります。

```bash
export PIPELINE_NAME="my_pipeline"
export PROJECT_DIR=~/tfx/${PIPELINE_NAME}
```

In [ ]:
PIPELINE_NAME="my_pipeline"
import os
# Create a project directory under Colab content directory.
PROJECT_DIR=os.path.join(os.sep,"content",PIPELINE_NAME)

TFX には、TFX python パッケージと `taxi` テンプレートが含まれています。分類や回帰など、ポイントワイズの予測問題を解決するには、このテンプレートを利用してスタートできます。

`tfx template copy` CLI コマンドは、事前定義されたテンプレートファイルをプロジェクトディレクトリにコピーします。

```sh
tfx template copy \
   --pipeline_name="${PIPELINE_NAME}" \
   --destination_path="${PROJECT_DIR}" \
   --model=taxi
```

In [ ]:
!tfx template copy \
  --pipeline_name={PIPELINE_NAME} \
  --destination_path={PROJECT_DIR} \
  --model=taxi

このノートブックの作業ディレクトリコンテキストをプロジェクトディレクトリに変更します。

```bash
cd ${PROJECT_DIR}
```

In [ ]:
%cd {PROJECT_DIR}

## ステップ 3. コピーしたソースファイルを閲覧する

TFX テンプレートは、Python ソースコード、サンプルデータ、パイプラインの出力を分析する Jupyter Notebook など、パイプラインを構築するための基本的なスキャフォールドファイルを提供します。`taxi` テンプレートは、<a>Airflow Tutorial </a>と同じ<em>シカゴタクシー</em>データセットと ML モデルを使用します。

Google Colab では、左側のフォルダアイコンをクリックしてファイルを参照できます。この場合、ファイルは `my_pipeline` という名前のプロジェクトディレクトリの下にコピーする必要があります。ディレクトリ名をクリックするとディレクトリの内容が表示され、ファイル名をダブルクリックして開くことができます。

ここでは、各 Python ファイルを簡単に紹介します。

- pipeline - このディレクトリには、パイプラインの定義が含まれています。
    - configs.py — パイプライン ランナーの共通定数を定義します。
    - pipeline.py — TFX コンポーネントとパイプラインを定義します。
- models - このディレクトリには、機械学習モデルの定義が含まれています。
    - `features.py`、`features_test.py` — モデルの特徴を定義します。
    - `preprocessing.py`、`preprocessing_test.py` — `tf::Transform`を使用して前処理ジョブを定義します。
    - estimator - このディレクトリには、Estimator ベースのモデルが含まれています。
        - constants.py — モデルの定数を定義します。
        - `model.py`、`model_test.py` — TF estimator を使用して DNN モデルを定義します
    - keras - このディレクトリには、Keras ベースのモデルが含まれています。
        - constants.py — モデルの定数を定義します。
        - `model.py`、`model_test.py` — Keras を使用して DNN モデルを定義します。
- `beam_runner.py`、`kubeflow_runner.py` — オーケストレーション エンジンごとにランナーを定義します。


名前に `_test.py` が含まれているファイルがあることに気が付きましたか。これらはパイプラインの単体テストであり、独自のパイプラインを実装するときに単体テストを追加することをお勧めします。テストファイルのモジュール名に `-m` フラグを指定すると、単体テストを実行できます。通常、モジュール名を取得するには、`.py` 拡張子を削除し、`/` を `.` に置き換えます。以下に例を示します。

```bash
python -m models.features_test
```

In [ ]:
!{sys.executable} -m models.features_test
!{sys.executable} -m models.keras.model_test


## ステップ 4. 最初の TFX パイプラインを実行する

`pipeline create` コマンドを使用してパイプラインを作成できます。

```bash
tfx pipeline create --engine=local --pipeline_path=local_runner.py
```


In [ ]:
!tfx pipeline create --engine=local --pipeline_path=local_runner.py

次に、`run create` コマンドを使用して作成されたパイプラインを実行できます。

```sh
tfx run create --engine=local --pipeline_name="${PIPELINE_NAME}"
```

In [ ]:
!tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

成功すると、`Component CsvExampleGen is finished` と表示されます。テンプレートをコピーする場合、パイプラインに含まれるコンポーネントは CsvExampleGen の 1 つだけです。

## ステップ 5. データ検証用のコンポーネントを追加する

このステップでは、`StatisticsGen`、`SchemaGen`、および、`ExampleValidator` などのデータ検証用のコンポーネントを追加します。データ検証については、[Tensorflow データ検証の基礎](https://www.tensorflow.org/tfx/data_validation/get_started)を参照してください。

コピーされたパイプライン定義を `pipeline/pipeline.py` で変更します。ローカル環境で作業している場合は、お気に入りのエディタを使用してファイルを編集します。Google Colab で作業している場合は、次を実行します。

> **左側のフォルダアイコンをクリックして、 `Files` ビューを開きます**。

> **`my_pipeline` をクリックしてディレクトリを開き、`pipeline` ディレクトリをクリックして開き、`pipeline.py` をダブルクリックしてファイルを開きます**。

> `StatisticsGen`、`SchemaGen`、および `ExampleValidator` をパイプラインに追加する 3 行を見つけてコメントを外します。（ヒント: `TODO(step 5):` が含まれているコメントを見つけます。

> 変更は数秒で自動的に保存されます。 `pipeline.py` の前にある `*` マークがタブタイトルに表示されていないことを確認してください。**Colab にはファイルエディタの保存ボタンやショートカットはありません。ファイルエディタの Python ファイルは、`playground` モードでもランタイム環境に保存できます。**

次に、パイプライン定義を変更して既存のパイプラインを更新します。`tfx pipeline update` コマンドを使用してパイプラインを更新し、続いて `tfx run create` コマンドを使用して更新されたパイプラインの新しい実行を作成します。

```sh
# Update the pipeline
tfx pipeline update --engine=local --pipeline_path=local_runner.py
# You can run the pipeline the same way.
tfx run create --engine local --pipeline_name "${PIPELINE_NAME}"
```


In [ ]:
# Update the pipeline
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
# You can run the pipeline the same way.
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

追加されたコンポーネントからの出力ログを確認できるはずです。パイプラインは、`tfx_pipeline_output/my_pipeline` ディレクトリに出力アーティファクトを作成します。

## ステップ 6. トレーニング用のコンポーネントを追加する

このステップでは、`Transform`、`Trainer`、`Resolver`、`Evaluator`、および、`Pusher` などのトレーニングとモデル検証用のコンポーネントを追加します。

> **`pipeline/pipeline.py`を開きます**。パイプラインに`Transform`、`Trainer`、`Resolver`、`Evaluator`、`Pusher` を追加する 5 行を見つけてコメントを外します（ヒント: `TODO(step 6):` を見つけます）。

以前と同様に、変更されたパイプライン定義で既存のパイプラインを更新する必要があります。手順はステップ 5 と同じです。`tfx pipeline update` を使用してパイプラインを更新し、`tfx run create` を使用して実行を作成します。

```sh
tfx pipeline update --engine=local --pipeline_path=local_runner.py
tfx run create --engine local --pipeline_name "${PIPELINE_NAME}"
```

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

この実行が正常に終了すると、ローカルオーケストレーターを使用して最初の TFX パイプラインを作成して実行できます。

**注意：**パイプライン実行を作成するたびに、入力とパラメーターが変更されていなくても、すべてのコンポーネントが何度も実行されることに気付いたかもしれません。これは時間とリソースの無駄なので、パイプラインキャッシングを使用してこれらの実行をスキップできます。`pipeline.py` の `Pipeline` オブジェクトに `enable_cache=True` を指定することで、キャッシュを有効にできます。


## ステップ 7.（*オプション*）BigQueryExampleGen を試してみる

BigQuery は、サーバーレスでスケーラビリティと費用対効果の高いクラウド データ ウェアハウスです。BigQuery は、TFX のトレーニング サンプルのソースとして使用できます。このステップでは、パイプラインにBigQueryExampleGenを追加します。

BigQuery を使用するには、[Google Cloud Platform](https://cloud.google.com/gcp/getting-started) アカウントが必要です。GCP プロジェクトを準備してください。

colab 認証ライブラリまたは `gcloud` ユーティリティを使用してプロジェクトにログインします。

```sh
# You need `gcloud` tool to login in local shell environment.
gcloud auth login
```

In [ ]:
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')


TFX を使用して BigQuery リソースにアクセスするには、GCP プロジェクト名を指定する必要があります。`GOOGLE_CLOUD_PROJECT` 環境変数をプロジェクト名に設定します。

```sh
export GOOGLE_CLOUD_PROJECT=YOUR_PROJECT_NAME_HERE
```

In [ ]:
# Set your project name below.
# WARNING! ENTER your project name before running this cell.
%env GOOGLE_CLOUD_PROJECT=YOUR_PROJECT_NAME_HERE

> **`pipeline/pipeline.py` を開きます**。`CsvExampleGen` をコメントアウトし、`BigQueryExampleGen` のインスタンスを作成する行のコメントを外します。また、`create_pipeline` 関数の `query` 引数のコメントも外す必要があります。

BigQuery に使用する GCP プロジェクトを再度指定する必要があります。そのためには、パイプラインの作成時に `beam_pipeline_args` に `--project` を設定します。

> **`pipeline/configs.py` を開きます**。`BIG_QUERY__WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` と `BIG_QUERY_QUERY` の定義のコメントを外します。このファイルのプロジェクト ID とリージョンの値を GCP プロジェクトの正しい値に置き換える必要があります。

> **`local_runner.py` を開きます**。create_pipeline() メソッドの 2 つの引数、`query` と `beam_pipeline_args` のコメントを外します。

これで、パイプラインでサンプルソースとして BigQuery を使用する準備が整いました。ステップ 5 と 6 で行ったように、パイプラインを更新して実行を作成します。

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

## 次のステップ: データをパイプラインに取り込む

シカゴタクシーデータセットを使用してモデルのパイプラインを作成しました。次に、データをパイプラインに入れます。

データは、GCS や BigQuery など、パイプラインがアクセスできる場所であればどこにでも保存できます。データにアクセスするには、パイプライン定義を変更する必要があります。

1. データがファイルに保存されている場合は、`kubeflow_runner.py` または `local_runner.py` の `DATA_PATH` を変更し、ファイルの場所に設定します。データが BigQuery に保存されている場合は、`pipeline/configs.py` の `BIG_QUERY_QUERY` を変更して、データを正しくクエリします。
2. models/features.pyに特徴量を追加します。
3. models/preprocessing.pyを変更して、トレーニング用の入力データを変換します。
4. models/keras/model.pyとmodels/keras/constants.pyを変更して、機械学習モデルを記述します。
    - Estimator ベースのモデルを使用することもできます。 `pipeline/configs.py` の `RUN_FN` 定数を `models.estimator.model.run_fn` に変更します。

詳細については、[トレーナーコンポーネントガイド](https://www.tensorflow.org/tfx/guide/trainer)を参照してください。